<a href="https://colab.research.google.com/github/Hamza-333/Text-Summarizer/blob/main/413_Final_Project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from datasets import list_datasets, load_dataset

In [4]:
datasets = list_datasets()

In [5]:
# load cnn-daily mail
# cnn = load_dataset('cnn_dailymail', '1.0.0')

In [6]:
# cnn['train'][0]

In [7]:
# load multi news
# multi_news = load_dataset('multi_news')

In [8]:
# # load stacked sum
# stacked_sum = load_dataset('stacked-summaries/stacked-xsum')

In [9]:
# using around 100k train, 19k validation, 1.95k test
gigaword = load_dataset('gigaword', split = ['train[:3%]', 'validation[:10%]', 'test'])

  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
# rename columns to have same column names across datasets
# cnn = cnn.rename_column("highlights", 'summary')
# cnn = cnn.rename_column("article", 'document')

In [11]:
# delete irrelevant columns
# cnn = cnn.remove_columns("id")
# stacked_sum = stacked_sum.remove_columns(['id','chapter_length', 'summary_length',\
#                             'is_stacked'])

In [12]:
from datasets.combine import concatenate_datasets

In [13]:
# concatenate all datasets
# train_data = concatenate_datasets([cnn['train'], multi_news['train']
#                                   ])
# val_data = concatenate_datasets([cnn['validation'], multi_news['validation']])
# test_data = concatenate_datasets([cnn['test'], multi_news['test']])

In [14]:
# train_data = cnn['train']
# val_data = cnn['validation']
# test_data = cnn['test']

train_data = gigaword[0]
val_data = gigaword[1]
test_data = gigaword[2]

In [15]:
# save data so we don't have to redownload every time

# train_data.to_csv('train.csv')
# val_data.to_csv('val.csv')
# test_data.to_csv('test.csv')

In [16]:
train_data[0]

{'document': "australia 's current account deficit shrunk by a record #.## billion dollars -lrb- #.## billion us -rrb- in the june quarter due to soaring commodity prices , figures released monday showed .",
 'summary': 'australian current account deficit narrows sharply'}

# Data Preparation

In [16]:
import torch

In [18]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.9 MB/s eta 0:00:00


In [17]:
from transformers import AutoTokenizer

In [20]:
train_data[0]['document']

'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details o

In [18]:
tokenizer = AutoTokenizer.from_pretrained("gpt2", special_tokens=[['BOS']])
tokenizer.pad_token = tokenizer.eos_token
device = "cuda:0" if torch.cuda.is_available() else "cpu"


In [19]:
inp_lst = []


In [20]:
def preprocess_function(examples):
    """Create tokens for each example. Padding is done using the max length of
    example in the batch. Each example in the batch will have equal length"""
    inputs = [doc for doc in examples['document']]
    model_inputs = tokenizer(inputs, padding=True, return_tensors="pt")
    # model_inputs = tokenizer.convert_to_indices(examples['text'])

    labels = tokenizer(text_target=examples["summary"], padding=True, return_tensors='pt')

    
    model_inputs["labels"] = labels["input_ids"]
    # store as pairs of documents and summaries
    # store the vocabulary indexes of each word
    inp_lst.append((model_inputs["input_ids"], labels["input_ids"]))

    return model_inputs

In [21]:
import torch.nn as nn

In [23]:
# preprocessing function tokenizes in batches
# inp_lst stores each batch
train_data = train_data.map(preprocess_function, batched = True, batch_size = 16)

Map:   0%|          | 0/114119 [00:00<?, ? examples/s]

In [24]:
vocab_size = len(tokenizer.vocab)

In [25]:
EMB_DIM = 300

In [26]:
import math

In [27]:
class PositionalEncoding(nn.Module):
    def __init__(self):
        super(PositionalEncoding, self).__init__()

    def positional_encoding(self, dim_model, length):
        pos_encod = torch.zeros(length, dim_model)
        pos_list = torch.arange(0, length).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-math.log(10000.0)) / dim_model) #1000^(2i/dim_model)

        pos_encod[:, 0::2] = torch.sin(pos_list * div_term) # pe(even) = sin(pos/1000^(2i/dim_model))
        pos_encod[:, 1::2] = torch.cos(pos_list * div_term) # pe(odd) = cos(pos/1000^(2i/dim_model))
        if torch.cuda.is_available():
            pos_encod= pos_encod.cuda()
        return pos_encod
    
    def forward(self, embedding, dim_model, length):
        return embedding + self.positional_encoding(dim_model, length)


In [28]:
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=EMB_DIM, nhead=20, num_encoder_layers=16, num_decoder_layers=16,
                     dim_feedforward=4096, dropout=0.1):
        super(TransformerModel, self).__init__()

        self.pos = PositionalEncoding()
        # transformer model with encoder and decoders
        self.transformer = nn.Transformer(d_model, nhead, num_encoder_layers, num_decoder_layers,
                dim_feedforward, dropout, activation='relu', layer_norm_eps=1e-05, batch_first=True)
        if torch.cuda.is_available():
            self.transformer.cuda()
        # final fully connected linear layer for classification
        self.fc = nn.Linear(d_model, vocab_size)



    def forward(self, inp, tgt):

        # Positional encoding for input and target

        inp = self.pos(inp, EMB_DIM, inp.size(1))
        tgt = self.pos(tgt, EMB_DIM, tgt.size(1))
        # target mask for decoder
        tgt_mask = self.transformer.generate_square_subsequent_mask(tgt.size(1), device='cuda')
        # encoder and decoder of transformer
        out = self.transformer(inp, tgt, tgt_mask=tgt_mask)

        # final linear layer
        out = self.fc(out)

        return out

    

In [29]:
import torch.optim as optim

In [30]:
import random

In [31]:
from torchtext.vocab import GloVe
glove = GloVe(name='42B', dim=300)

.vector_cache/glove.42B.300d.zip: 1.88GB [05:54, 5.30MB/s]                            
100%|█████████▉| 1917493/1917494 [03:23<00:00, 9407.49it/s]


In [12]:
import pickle

In [36]:

from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [58]:
# save tokens
# with open("/content/gdrive/My Drive/inputs.pkl", "wb") as fp:   
#     pickle.dump(inp_lst, fp)

In [13]:
# Only run if you have saved the file
with open("/content/gdrive/My Drive/inputs.pkl", "rb") as fp: 
    inp_lst = pickle.load(fp)

In [37]:
def glove_embed(data):
    """Get glove embeddings of sequences"""
    max_length = 0 
    # find the maximum length of a sequence in the batch
    # need to pad every sequence by that much
    for i in range(len(data)):
        max_length = max(max_length, len(data[i]))
    # create empty tensor of size len(data), max_length, size of glove embedding

    tensor = torch.empty(len(data), max_length, EMB_DIM)
    for i in range(len(data)):
        # get glove embedding, use lower case back up if token not found
        words = tokenizer.convert_ids_to_tokens(data[i])
        emb = glove.get_vecs_by_tokens(words, lower_case_backup=True)
        tensor[i] = emb

    return tensor

In [19]:
import gc

In [43]:
def train(model, train_data, batch_size=16, weight_decay=0.0,
           learning_rate=0.0000001, num_epochs=20, checkpoint_path=None):

    model.train()
    # ignore padding tokens in loss computation
    criterion = nn.CrossEntropyLoss(ignore_index=50256)

    epoch = 0
    train_accs, train_losses = [], []
    val_accs = []
    iters = []
    n = 0
    epochs = []
    t_loss = 0
    if torch.cuda.is_available():
        model.cuda()
        criterion.cuda()
 
    batch = 0
    while epoch < num_epochs:
        random.shuffle(inp_lst)
        learning_rate = learning_rate
        optimizer = optim.AdamW(model.parameters(),
                lr=learning_rate,
                weight_decay=weight_decay)
        
        i = 1
        t_loss = 0
        for inp, tgt_ind in train_data:

            if len(inp) < batch_size:
                # Last batch will have less images since it will be the
                # remainder of Total images / batch_size.

                # Adjust the learning rate of the last batch by 
                # (size(last_batch) / batch_size) to account 
                # for the smaller size.
                learning_rate = learning_rate * (len(inp) / batch_size)
                optimizer = optim.AdamW(model.parameters(),
                    lr=learning_rate,
                    weight_decay=weight_decay)

            if torch.cuda.is_available():
                inp = inp.cuda()
                tgt_ind = tgt_ind.cuda()
            # print(inp.device)
            device = "cuda:0" if torch.cuda.is_available() else "cpu"
            # get glove embeddings
            inp = glove_embed(inp).to(device)
            tgt = glove_embed(tgt_ind).to(device)
            # with torch.autocast(device_type='cuda', dtype=torch.float16):


            outputs = model(inp, tgt)

            outputs = torch.permute(outputs, (0,2,1))
            # normalize loss
            
            loss = criterion(outputs, tgt_ind) / 16
            # print(loss)
            loss.backward()
            n += 1
            # print(outputs.size())
            
            # update parameters every fourth iteration so we can accumalte gradients
            # over smaller batches --> conserves memory

            if i % 16 == 0 or (i == len(train_data)):
                # print(i)
                optimizer.step()
                # j = 0
                # for param in model.parameters():
                #     j += 1
                #     if j == 2:
                #         print(param.data)

                #         break
                optimizer.zero_grad()
                t_loss += float(loss)/(batch_size)
                # get_summary(torch.permute(outputs, (0,2,1))[0])
                # get_summary(outputs[0])
                # print('---------------------------------------')
                # get_summary(torch.permute(outputs, (0,2,1))[0])
                # print("FIrst--------", loss)
            i += 1
            # if i % 100 == 0:
                
            if i % 1000 == 0:
                get_summary(torch.permute(outputs, (0,2,1))[0])
                print(i)
            
            iters.append(n)
            # gc.collect()
            # average loss
            
            # train_losses.append(t_loss)

        # end of epoch
        # break

        epoch += 1
        torch.save(model.state_dict(), checkpoint_path.format(epoch))
        print("Train loss: {} -----------------  epoch: {}".format(t_loss, epoch))
        

In [34]:
# vocab_size=50257

In [39]:
model = TransformerModel(vocab_size=vocab_size)

In [42]:
train(model, inp_lst, checkpoint_path='/content/gdrive/My Drive/CSC413/A2/Transformerckpt-{}.pk')

KeyboardInterrupt: ignored

In [145]:
# check output of the model
model.eval()
g1 = glove_embed(inp_lst[3][0]).to(device)
g2 = glove_embed(inp_lst[3][1]).to(device)
summary = model(g1, g2)

In [41]:
def get_summary(output):
    lst = []
    # for i in range(output.size(0)):
    pred = output.max(1, keepdim=True)[1]

    # break
    words = tokenizer.convert_ids_to_tokens(pred)
    w = tokenizer.decode(pred[0])
    # lst.append(words)
    print(words)

In [ ]:
tokenizer.convert_ids_to_tokens(inp_lst[4][0][0])

In [55]:
summary.size()

torch.Size([16, 14, 50257])

In [140]:
get_summary(summary)

['Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto']
['Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto']
['Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto']
['Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto']
['Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto']
['Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto']
['Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto']
['Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto']
['Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto', 'Ġto

In [ ]:
tokenizer.decode(summary)

In [26]:
test_data = test_data.map(preprocess_function, batched = True)

In [ ]:
test_data[:5]['input_ids']

**** DONT RUN ANYTHING UNDER THIS --------------------------------------------------------------** **

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# !wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
# !bash Miniconda3-latest-Linux-x86_64.sh

In [ ]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()

In [ ]:
# !conda --version

conda 22.11.1


In [ ]:
#DONT RUN
# !conda install -c nvidia/label/cuda10.0 -c rapidsai/label/cuda10.0 -c numba -c conda-forge -c defaults cudf

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 

In [ ]:
# restart runtime after installing
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

In [ ]:
import torch

import cudf
from cudf.utils.hash_vocab_utils import hash_vocab


from cudf.core.subword_tokenizer import SubwordTokenizer

/usr/local/lib/python3.9/dist-packages/rmm/__init__.py:56: FutureWarning: Use of 'rmm.RMMNumbaManager' is deprecated and will be removed. 'RMMNumbaManager' now lives in the 'rmm.allocators.numba' sub-module, please update your imports.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/rmm/__init__.py:56: FutureWarning: Use of 'rmm.rmm_cupy_allocator' is deprecated and will be removed. 'rmm_cupy_allocator' now lives in the 'rmm.allocators.cupy' sub-module, please update your imports.
  warnings.warn(


In [ ]:
import pandas as pd

In [ ]:
# convert train_data to pandas dataframe
train_data.set_format("pandas")

In [ ]:
val_data.set_format("pandas")
test_data.set_format("pandas")

In [ ]:
def convert_to_pandas(data):
    # Convert in batches to preserve memory
    # Use only 1,000,000 data points for now
    batch_size = 50000
    df = data[0]
    batch = 1
    for i in range(1, 20):

        tmp = data[batch:batch+batch_size]
        batch += batch_size
        df = pd.concat([df, tmp], axis=0)
    return df
    

In [ ]:
train_df = convert_to_pandas(train_data)

In [ ]:
val_df = convert_to_pandas(val_data)
test_df = convert_to_pandas(test_data)

In [ ]:
import gc
gc.collect()

In [ ]:
def convert_to_cudf(df):
    # convert from pandas to cudf to speed up processing
    # cudf uses gpu to tokenize data
    # convert in batches to preserve memory
    batch_size = 50000
    cdf = cudf.from_pandas(df[:batch_size])

    batch = 1
    j = 2
    for i in range(1, 20):

        tmp = cudf.from_pandas(df[batch:batch+batch_size])

        batch += batch_size
        j += 1
        cdf = cudf.concat([cdf.copy(), tmp.copy()], axis=0)
    return cdf

In [ ]:
train_cdf = convert_to_cudf(train_df)

In [ ]:
val_cdf = convert_to_cudf(val_df)
test_cdf = convert_to_cudf(test_df)

In [ ]:
from collections import Counter

In [ ]:
from torchtext.vocab import vocab

In [ ]:
def build_vocab(df):
    """Build vocab objects for both document and summary"""
    # vocabulary counters
    tokens_doc = Counter()
    tokens_sum = Counter()
    # tokenize each column
    t_doc = df['document'].str.tokenize().to_pandas()
    t_sum = df['summary'].str.tokenize().to_pandas()

    tokens_doc.update(t_doc.values)
    tokens_sum.update(t_sum.values)


    doc_vocab = vocab(tokens_doc, specials=['<unk>', '<pad>', '<bos>', '<eos>'])
    summary_vocab = vocab(tokens_sum, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

    return t_doc, t_sum, doc_vocab, summary_vocab



In [ ]:
# training vocab and tokens
t_doc, t_sum, doc_vocab, sum_vocab = build_vocab(train_cdf)

In [ ]:
val_cdf.head(5)

,document,summary
0,"(CNN)Share, and your gift will be multiplied. ...",Zully Broussard decided to give a kidney to a ...
0,"(CNN)On the 6th of April 1996, San Jose Clash ...",The 20th MLS season begins this weekend . Leag...
1,"(CNN)French striker Bafetimbi Gomis, who has a...",Bafetimbi Gomis collapses within 10 minutes of...
2,(CNN)It was an act of frustration perhaps more...,Rory McIlroy throws club into water at WGC Cad...
3,(CNN)A Pennsylvania community is pulling toget...,"Cayman Naib, 13, hasn't been heard from since ..."


In [ ]:
# val tokens
# do not need val vocab

val_t_doc, val_t_sum, val_doc_vocab, val_sum_vocab = build_vocab(val_cdf)

In [ ]:
# test tokens
# do not need test vocab

test_t_doc, test_t_sum, test_doc_vocab, test_sum_vocab = build_vocab(test_cdf)

In [ ]:
train_cdf['document'][0]

In [ ]:
train_cdf.reset_index(inplace=True)

In [ ]:
val_cdf.reset_index(inplace=True)
test_cdf.reset_index(inplace=True)

In [ ]:
def tokenize(cdf, tokens, col):
    """Create a combines list of tokens"""
    i = 0
    lst = []
    j = 0
    total = 0
    while j < len(cdf):
        n = len(cdf[col][j])
        total += n
        # store tokenized representation of each text
        lst.append(tokens[i:total].values)
        i += n 
        j += 1
    return lst

In [ ]:
# training document tokens
tokenized_docs = tokenize(train_cdf, t_doc, 'document')

In [ ]:
# training summary tokens
tokenized_sums = tokenize(train_cdf, t_sum, 'summary')

In [ ]:
# get tokenized validation documents and summaries
val_tok_docs = tokenize(val_cdf, val_t_doc, 'document')
val_tok_sums = tokenize(val_cdf, val_t_sum, 'summary')

In [ ]:
# get tokenized test documents and summaries
test_tok_docs = tokenize(test_cdf, test_t_doc, 'document')
test_tok_sums = tokenize(test_cdf, test_t_sum, 'summary')

In [ ]:
test_tok_sums

In [1]:
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
# save train tokens in file so we don't have to run it again.
with open("/content/gdrive/My Drive/tokenized_doc.pkl", "wb") as fp:   
    pickle.dump(tokenized_docs, fp)

In [3]:
# load the tokens
with open("/content/gdrive/My Drive/tokenized_doc.pkl", "rb") as fp: 
    train_docs = pickle.load(fp)

In [ ]:
with open("/content/gdrive/My Drive/tokenized_sum.pkl", "wb") as fp:   
    pickle.dump(tokenized_sums, fp)

In [4]:
with open("/content/gdrive/My Drive/tokenized_sum.pkl", "rb") as fp: 
    train_sums = pickle.load(fp)

In [ ]:
# save val tokens in file
with open("/content/gdrive/My Drive/val_tok_doc.pkl", "wb") as fp:   
    pickle.dump(val_tok_docs, fp)
with open("/content/gdrive/My Drive/val_tok_sum.pkl", "wb") as fp:   
    pickle.dump(val_tok_sums, fp)

In [ ]:
# load val tokens
with open("/content/gdrive/My Drive/val_tok_doc.pkl", "rb") as fp: 
    val_docs = pickle.load(fp)
with open("/content/gdrive/My Drive/val_tok_sum.pkl", "rb") as fp: 
    val_sums = pickle.load(fp)

In [ ]:
# save test tokens in file
with open("/content/gdrive/My Drive/test_tok_doc.pkl", "wb") as fp:   
    pickle.dump(test_tok_docs, fp)
with open("/content/gdrive/My Drive/test_tok_sum.pkl", "wb") as fp:   
    pickle.dump(test_tok_sums, fp)

In [ ]:
# load test tokens
with open("/content/gdrive/My Drive/test_tok_doc.pkl", "rb") as fp: 
    test_docs = pickle.load(fp)
with open("/content/gdrive/My Drive/test_tok_sum.pkl", "rb") as fp: 
    test_sums = pickle.load(fp)

In [5]:
import torch

In [ ]:
torch.save(sum_vocab, '/content/gdrive/My Drive/sum_vocab.pth')
torch.save(doc_vocab, '/content/gdrive/My Drive/doc_vocab.pth')

In [6]:
sum_vocab =torch.load('/content/gdrive/My Drive/sum_vocab.pth')
doc_vocab =torch.load('/content/gdrive/My Drive/doc_vocab.pth')

In [7]:
import gc
gc.collect()

49

# Build Model

In [28]:
from torchtext.vocab import GloVe

In [32]:
import torch.nn as nn

In [33]:
class PositionalEncoding(nn.Module):
    def __init__(self):
        super(PositionalEncoding, self).__init__()

    def positional_encoding(self, dim_model, length):
        pos_encod = torch.zeros(length, dim_model)
        pos_list = torch.arange(0, length).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-math.log(10000.0)) / dim_model) #1000^(2i/dim_model)

        pos_encod[:, 0::2] = torch.sin(pos_list * div_term) # pe(even) = sin(pos/1000^(2i/dim_model))
        pos_encod[:, 1::2] = torch.cos(pos_list * div_term) # pe(odd) = cos(pos/1000^(2i/dim_model))

        return pos_encod
    
    def forward(self, embedding, dim_model, length):
        return embedding + self.positional_encoding(dim_model, length)


In [41]:
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=64, nhead=8, num_encoder_layers=3, num_decoder_layers=3,
                     dim_feedforward=2048, dropout=0.1):
        super(TransformerModel, self).__init__()

        # glove embedding layer
        # self.glove = GloVe(name='6B', dim=50)
        self.emb = nn.Embedding(vocab_size, 64)
        # positonal encoding layer
        self.pos = PositionalEncoding()
        # transformer model with encoder and decoders
        self.transformer = torch.nn.Transformer(d_model, nhead, num_encoder_layers, num_decoder_layers,
                dim_feedforward, dropout, activation='relu', layer_norm_eps=1e-05)
        if torch.cuda.is_available():
            self.transformer.cuda()
        # final fully connected linear layer for classification
        self.fc = nn.Linear(d_model, vocab_size)



    def forward(self, inp, tgt):
        # glove embed inputs and targets
        inp = self.emb(inp)
        tgt = self.emb(tgt)

        # convert to tensor

        # Positional encoding for input and target
        inp = self.pos(inp)
        tgt = self.pos(tgt)
        # target mask for decoder
        tgt_mask = self.transformer.get_square_subsequent_mask(tgt.size(0))
        # encoder and decoder of transformer
        out = self.transformer(inp, tgt, tgt_mask=tgt_mask)
        # final linear layer
        out = self.fc(out)

        return out

    

In [12]:
import numpy as np

# Training


In [ ]:
!pip install numba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from numba import jit

In [49]:
@jit(parallel=True)
def add_tokens(data, max_length):
    """Add EOS, BOS tokens and padding '<pad>', '<bos>', '<eos>'"""
    # data = data.astype(str)
    # for i in range(len(data)):
        # data[i] = list(data[i])
    print(type(data[0]))
    print(data[0])
    if data[0] != '<bos>':
        print("h")
        data = np.insert(data, 0, '<bos>')
        data = np.append(data, '<eos>')
    
    #     data[i].insert(0, '<bos>')
    # data[i].append('<eos>')
    # calculate the amount needed to pad
    # add two to account for '<bos>', '<eos>' tokens
    diff = max_length - len(data) + 2

    # data[i].extend(['<pad>'] * diff)
    data = np.concatenate((data, np.array(['<pad>'] * diff)))
    # print(max_length)
    # print(data[i])
    gc.collect()
    # break
    # if i % 1000 == 0:
    
    return data

In [ ]:
@jit
def glove_embed(data, glove, emb_dim, max_length):
    """Get glove embeddings of sequences"""
    # create empty tensor of size len(data), max_length, size of glove embedding
    tensor = torch.empty(len(data), max_length, emb_dim)
    for i in range(len(data)):
        # get glove embedding, use lower case back up if token not found
        emb = glove.get_vecs_by_tokens(data[i], lower_case_backup=True)
        tensor[i] = emb

    return tensor

In [ ]:
@jit
def get_indices(data, v):
    """Get vocabulary index of each word in the sequences"""
    indices = []
    for seq in data:
        indices.append(v.lookup_indices(list(seq)))
    return torch.tensor(indices)

In [35]:
def max_len(data):
    max_length = 0 
    # find the maximum length of a sequence in the batch
    # need to pad every sequence by that much
    for i in range(len(data)):
        max_length = max(max_length, len(data[i]))
    return max_length

In [19]:
import torch.optim as optim

In [20]:
from torchtext.vocab import GloVe
glove = GloVe(name='6B', dim=50)

.vector_cache/glove.6B.zip: 862MB [02:41, 5.35MB/s]                           
100%|█████████▉| 399999/400000 [00:10<00:00, 38443.39it/s]


In [ ]:
!pip install pycuda

In [ ]:
import pycuda.autoinit
import pycuda.driver as cuda

In [ ]:
import pycuda.gpuarray as gpu

In [58]:
def train(model, glove, train_docs, train_sums, batch_size=4096, weight_decay=0.0,
           learning_rate=0.001, num_epochs=7):

    model.train()
    criterion = nn.CrossEntropyLoss()

    epoch = 0
    train_accs, train_losses = [], []
    val_accs = []
    iters = []
    n = 0
    epochs = []
    N = len(train_docs)
    if torch.cuda.is_available():
        model.cuda()
        criterion.cuda()
 
    batch = 0
    while epoch < num_epochs:
 
        learning_rate = learning_rate
        optimizer = optim.Adam(model.parameters(),
                lr=learning_rate,
                weight_decay=weight_decay)
        
        # perms = torch.randperm(N)
        while batch < N:
            # ind = perms[batch: batch + batch_size]
            # get batch of documents and summaries
            inp = np.array(train_docs[batch: batch + batch_size],dtype=object)
            tgt = np.array(train_sums[batch: batch + batch_size],dtype=object)
            
            print(inp.shape)
            # i_gpu = cuda.mem_alloc(inp.nbytes)
            # cuda.memcpy_htod(i_gpu, inp)
            # inp_gpu = gpu.to_gpu(inp)
            # print(inp_gpu)
            # # t_gpu = cuda.mem_alloc(tgt.nbytes)
            # # cuda.memcpy_htod(t_gpu, tgt)
            # tgt_gpu = gpu.to_gpu(tgt)
            
            batch += batch_size
        
            if len(inp) < batch_size:
                # Last batch will have less images since it will be the
                # remainder of Total images / batch_size.

                # Adjust the learning rate of the last batch by 
                # (size(last_batch) / batch_size) to account 
                # for the smaller size.
                learning_rate = learning_rate * (len(inp) / batch_size)
                optimizer = optim.Adam(model.parameters(),
                    lr=learning_rate,
                    weight_decay=weight_decay)
            max_length_inp = max_len(inp)
            max_length_tgt = max_len(tgt)
            # add padding and eos and bos tokens
            inp = np.apply_along_axis(add_tokens, arr=inp, axis=0, max_length=max_length_inp)
            tgt = add_tokens(tgt, max_length_tgt)
            # expected output in terms of vocabulary indices
            # need this for loss function
            exp_outputs = get_indices(tgt, sum_vocab)
            # get glove embeddings
            inp = glove_embed(train_docs, glove, 50, max_length_inp)
            tgt = glove_embed(train_sums, glove, 50, max_length_tgt)
            if torch.cuda.is_available():
                inp = inp.cuda()
                tgt = tgt.cuda()
                exp_outputs = exp_outputs.cuda()

            outputs = model(inp, tgt)
            loss = criterion(outputs, exp_outputs)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            n += 1
            iters.append(n)
            # average loss
            t_loss = float(loss)/batch_size
            train_losses.append(t_loss)

        # end of epoch
        epoch += 1
        print("Train loss: {} -----------------  epoch: {}".format(t_loss, epoch))
        

In [22]:
model = TransformerModel(vocab_size=vocab_size)
train(model, glove, train_docs, train_sums)

In [59]:
train(model, glove, train_docs, train_sums)

(4096,)
<class 'numpy.ndarray'>
['LONDON,' 'England' '(Reuters)' ... 'in' '2004,' 'according']


ValueError: ignored

In [ ]:
len(sum_vocab)

693633

In [ ]:
train()

In [ ]:
import torch

def positional_encoding(dim_model, length):
  pos_encod = torch.zeros(length, dim_model)
  pos_list = torch.arange(0, length).unsqueeze(1)
  div_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-math.log(10000.0)) / dim_model) #1000^(2i/dim_model)

  pos_encod[:, 0::2] = torch.sin(pos_list * div_term) # pe(even) = sin(pos/1000^(2i/dim_model))
  pos_encod[:, 1::2] = torch.cos(pos_list * div_term) # pe(odd) = cos(pos/1000^(2i/dim_model))

  return pos_encod



In [ ]:
transformer_model = torch.nn.Transformer(d_model=50, nhead=8, num_encoder_layers=3, num_decoder_layers=3,
                     dim_feedforward=2048, dropout=0.1, activation="relu", layer_norm_eps=1e-05)

In [ ]:
# tokenize train data into document tokens and summary tokens
# merge into one dictionary
# get glove embeddings for each word
# Convert to tensors
# attach positional encodings
# Feed embeddings to encoder

